***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [37]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9074  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [38]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [40]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  7 09:13 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
!pip install numpy

In [41]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *
import numpy as np

In [42]:
spark

In [43]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '208899385irproject'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.endswith("preprocessed.parquet"):
        paths.append(full_path+b.name)
# print(paths)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

In [39]:
import pyspark
import sys
from collections import Counter, defaultdict
import nltk
from nltk.corpus import stopwords
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
from operator import add
import hashlib
from nltk.stem.porter import *
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [44]:
parquetFile = spark.read.parquet(*paths)

#rdd for each type of index that we need (title , text(body), anchor_text)
# limited = parquetFile.limit(1000)
# doc_title_pairs = parquetFile.limit(1000).select("title", "id").rdd
# doc_body_pairs = parquetFile.limit(1000).select("text", "id").rdd
# doc_anchor_pairs = parquetFile.limit(1000).select("anchor_text", "id").rdd
doc_title_pairs = parquetFile.select("title", "id").rdd
doc_body_pairs = parquetFile.select("text", "id").rdd
doc_anchor_pairs = parquetFile.select("anchor_text", "id").rdd


# need to do little manipulation on the anchor text rdd
def make_string_from_anchor(input_string :str) -> str:
    pattern = r"(?<=text=\')[^']+(?=\')"
    matches = re.findall(pattern, input_string)
    string_back = ""
    for term in matches:
        string_back += term + " "

    return string_back
doc_anchor_pairs = doc_anchor_pairs.map(lambda x: (Row(anchor_text=make_string_from_anchor(str(x[0]).lower()),id=x[1])))

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [45]:
# Count number of wiki pages
n = parquetFile.count()
print(n)

6348910


In [46]:
n=6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [47]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [48]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/07 16:35:07 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [49]:
from inverted_index_gcp import InvertedIndex

# Func for the implementation of the invertIndex

In [50]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
    ''' Count the frequency of each word in `text` (tf) that is not included in
    `all_stopwords` and return entries that will go into our posting lists.
    Parameters:
    -----------
      text: str
        Text of one document
      id: int
        Document id
    Returns:
    --------
      List of tuples
        A list of (token, (doc_id, tf)) pairs
        for example: [("Anarchism", (12, 5)), ...]
    '''
# Initialize the stemmer inside the function
    stemmer = PorterStemmer()

    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

    tokens = [stemmer.stem(token) for token in tokens]
    # Remove stopwords from the list of tokens
    filtered_tokens = [token for token in tokens if token not in all_stopwords]

    # Calculate the term frequency (tf) for each unique token
    tf_dict = {}
    for token in filtered_tokens:
        if token in tf_dict:
            tf_dict[token] += 1
        else:
            tf_dict[token] = 1

    # Create a list of tuples in the form (token, (doc_id, tf))
    result = [(token, (id, tf)) for token, tf in tf_dict.items()]
    
    return result

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
    Parameters:
    -----------
      unsorted_pl: list of tuples
        A list of (wiki_id, tf) tuples
    Returns:
    --------
      list of tuples
        A sorted posting list.
    '''
    if not unsorted_pl:
        return []

    # Sort the unsorted posting list by wiki_id
    sorted_pl = sorted(unsorted_pl)

    return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  df_rdd = postings.map(lambda x: (x[0], len(set(doc_id for doc_id, _ in x[1]))))

  return df_rdd


def partition_postings_and_write(postings, BASE_DIR):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''    # Function to write a posting list to disk
# Assign each posting to a specific bucket based on the token
  bucketed_postings = postings.map(lambda posting: (token2bucket_id(posting[0]), posting))
#   BASE_DIR = SparkFiles.getRootDirectory()
  # Organize postings by their bucket, accumulating postings lists for each bucket
  organized_by_bucket = bucketed_postings.groupByKey().map(lambda bucket_postings: (bucket_postings[0], list(bucket_postings[1])))
  # Function to process each bucket's postings and write to disk
  def process_and_write_bucket(bucket_with_posts):
        
      bucket_id, posts_list = bucket_with_posts
      # Format required by the write_a_posting_list method
      bucket_data = (bucket_id, posts_list)
      return InvertedIndex.write_a_posting_list(bucket_data,BASE_DIR,bucket_name)

  # Apply the writing function to each bucket and collect the posting locations
  locations_rdd = organized_by_bucket.map(process_and_write_bucket)

  return locations_rdd

# Create Title dict

In [51]:
# save in dict for each doc_id the title
title_dict = {}
for title, doc_id in doc_title_pairs.collect():
    title_dict[doc_id] = title


# Create doc_len dict

In [52]:
word = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
def get_doc_len(text_raw, doc_id):
    return doc_id, len(list(word.finditer(text_raw.lower())))

# save the len for each title:
pair = doc_title_pairs.map(lambda x : get_doc_len(x[0],x[1]))
title_len = Counter()
for doc_id,t_len in pair.collect():
    title_len[doc_id] = t_len

pair = doc_body_pairs.map(lambda x : get_doc_len(x[0],x[1]))
doc_len = Counter()
for doc_id,len_d in pair.collect():
    doc_len[doc_id] = len_d

with open("doc_len.pkl", 'wb') as f:
  pickle.dump(doc_len, f)
# print(doc_len[60283633])

# Reading from bucket index and read posting loc

In [53]:
from contextlib import closing
TUPLE_SIZE = 6

def read_posting_list(inverted, w, bucket_name):
  with closing(InvertedIndex.MultiFileReader()) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(bucket_name, locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

def read_pkl_file_form_bucket(file_name, name_bucket):
    """
        func that read pkl file from the bucket
    Args:
        name_bucket: name of the bucket
        file_name: the name of the pkl file + dir : pagerank\page_rank

    Returns:
            dict
    """
    # access to the bucket
    bucket = storage.Client().get_bucket(name_bucket)
    blob = bucket.get_blob(f'{file_name}.pkl')
    if blob:
      with blob.open("rb") as pkl_file:
          return pickle.load(pkl_file)

# Indexing Magic

In [64]:
def make_inverted_index(part_name: str) ->None:
#     BASE_DIR = SparkFiles.getRootDirectory()
    """
     func that create for each type : body , title .... inverter index
    Args:
      part_name: String body ,title ,anchor

    Returns: None

    """
    word_counts = None
    bucket_name_to_save = ""
    if part_name == "body":
      # word counts map
      bucket_name_to_save = bucket_name
      word_counts = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))
      BASE_DIR = 'posting_body'

    elif part_name == "title":
      # word counts map
      bucket_name_to_save = bucket_name
      word_counts = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
      BASE_DIR = 'posting_title'

    elif part_name == "anchor":
      # word counts map
      bucket_name_to_save = "/"
      word_counts = doc_anchor_pairs.flatMap(lambda x: word_count(x[0], x[1]))
    
    postings_filtered = word_counts.groupByKey().mapValues(reduce_word_counts)
    # filtering postings and calculate df
    if part_name == "body":
        # word counts map
        postings_filtered = postings_filtered.filter(lambda x: len(x[1])>50)

    w2df = calculate_df(postings_filtered)
    w2df_dict = w2df.collectAsMap()
       # Create inverted index instance
    inverted = InvertedIndex()
    inverted.term_total = postings_filtered.flatMapValues(lambda x : x).map(lambda x: (x[0],x[1][1])).reduceByKey(add).collectAsMap()
    if part_name == "body":
          inverted.avg_doc_len = np.sum(np.array(list(doc_len.values()))) / n
    elif part_name == "title":
          inverted.avg_doc_len = np.sum(np.array(list(title_len.values()))) / n




    # partition posting lists and write out
    _ = partition_postings_and_write(postings_filtered,BASE_DIR).collect()
    # collect all posting lists locations into one super-set
    super_posting_locs = defaultdict(list)
    for blob in client.list_blobs(bucket_name_to_save, prefix=BASE_DIR):
      if not blob.name.endswith("pickle"):
        continue
      with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
          super_posting_locs[k].extend(v)
    

    # adding the title of all the docs
    inverted.title = title_dict
    if part_name == "body":
        inverted.doc_len = doc_len
    # save the len of the entire corpus
    if part_name == "title":
        inverted.title_len = title_len  # Adding document lengths for body part
    inverted.n = n
    # Adding the posting locations dictionary to the inverted index
    inverted.posting_locs = super_posting_locs
    # Add the token - df dictionary to the inverted index
    inverted.df = w2df_dict
    # write the global stats out
    inverted.write_index('.', f'index_{part_name}')
    # upload to gs
    index_src = f"index_{part_name}.pkl"
    index_dst = f'gs://{bucket_name_to_save}/{BASE_DIR}/{index_src}'
    !gsutil cp $index_src $index_dst

In [65]:
# make all inverters index

make_inverted_index("title")
# make_inverted_index("anchor")
make_inverted_index("body")


Copying file://index_title.pkl [Content-Type=application/octet-stream]...
/ [1 files][119.6 KiB/119.6 KiB]                                                
Operation completed over 1 objects/119.6 KiB.                                    


Copying file://index_body.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 57.1 KiB/ 57.1 KiB]                                                
Operation completed over 1 objects/57.1 KiB.                                     


In [34]:
index_body = InvertedIndex.read_index('postings_gcp/',"index_body", bucket_name)
index_title = InvertedIndex.read_index('postings_gcp/',"index_title", bucket_name)
dox_len = InvertedIndex.read_index('',"doc_len", bucket_name)

TypeError: Invalid argument, not a string or column: dict_values([596, 410, 107, 2095, 18, 565, 231, 638, 837, 191, 195, 336, 137, 153, 241, 213, 81, 273, 50, 116, 296, 888, 149, 373, 56, 77, 118, 58, 311, 85, 54, 479, 26, 928, 45, 260, 33, 180, 39, 227, 926, 1001, 38, 422, 302, 209, 82, 211, 98, 37, 289, 334, 218, 55, 116, 504, 1198, 284, 531, 359, 74, 531, 233, 172, 455, 121, 462, 531, 234, 473, 47, 195, 455, 35, 561, 51, 335, 273, 60, 183, 24, 232, 1672, 136, 486, 251, 709, 42, 249, 139, 290, 72, 114, 286, 1444, 174, 119, 141, 2499, 69, 185, 298, 201, 789, 303, 1329, 74, 155, 185, 298, 339, 215, 209, 318, 235, 201, 194, 389, 399, 28, 130, 21, 54, 117, 128, 199, 222, 47, 70, 183, 570, 78, 120, 97, 77, 319, 167, 298, 314, 106, 32, 176, 284, 170, 450, 1446, 201, 33, 115, 101, 28, 575, 196, 604, 616, 344, 314, 76, 36, 73, 130, 168, 122, 1188, 3927, 46, 211, 77, 518, 82, 932, 132, 243, 557, 185, 225, 302, 165, 131, 192, 110, 408, 50, 151, 281, 92, 839, 649, 77, 40, 293, 68, 1573, 605, 47

# PageRank

In [33]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  edges= pages.flatMap(lambda k:map(lambda p:(k['id'],p[0]),k['anchor_text'])).distinct()
  vertices_1= pages.map(lambda k:k[0])
  vertices_2_anchor=pages.flatMap(lambda k:k[1])
  vertices_from_anchor= vertices_2_anchor.map(lambda k:k[0])
  vertices_union= vertices_1.union(vertices_from_anchor)
  vertices= vertices_union.distinct().map(lambda k:(k,))

  return edges, vertices

In [74]:
pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").limit(1000).select("id", "anchor_text").rdd

pages_links = spark.read.parquet(*paths).limit(1000).select("id", "anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
# pr = pr.sort(col('pagerank').desc())
# save the page rank dict
pr = pr.toPandas().to_dict()
pr = pd.DataFrame(pr)
dict_pr = dict(zip(pr.id, pr.pagerank))
#save the dict in pkl
with open('page_rank.pkl', 'wb') as file:
  pickle.dump(dict_pr, file)


24/03/06 15:01:13 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: gs://wikidata_preprocessed/*.
java.io.IOException: Error accessing gs://wikidata_preprocessed/*
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2231) ~[gcs-connector-hadoop3-2.2.20.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2121) ~[gcs-connector-hadoop3-2.2.20.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfoInternal(GoogleCloudStorageFileSystem.java:1141) ~[gcs-connector-hadoop3-2.2.20.jar:?]
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.getFileInfo(GoogleCloudStorageFileSystem.java:1115) ~[gcs-connector-hadoop3-2.2.20.jar:?]
	at com.google.cloud.hadoop.f

Py4JJavaError: An error occurred while calling o830.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 145.0 failed 4 times, most recent failure: Lost task 0.3 in stage 145.0 (TID 1988) (cluster-983c-w-2.c.phonic-sunbeam-414514.internal executor 14): java.io.IOException: Error accessing gs://wikidata_preprocessed/multistream10_preprocessed.parquet
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getObject(GoogleCloudStorageImpl.java:2231)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageImpl.getItemInfo(GoogleCloudStorageImpl.java:2121)
	at com.google.cloud.hadoop.repackaged.gcs.com.google.cloud.hadoop.gcsio.GoogleCloudStorageFileSystem.listFileInfo(GoogleCloudStorageFileSystem.java:1072)
	at com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystemBase.listStatus(GoogleHadoopFileSystemBase.java:957)
	at org.apache.spark.util.HadoopFSUtils$.listLeafFiles(Hadoo

# Page view

In [ ]:
# Paths
# Using user page views (as opposed to spiders and automated traffic) for the
# month of August 2021
pv_path = 'https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2'
p = Path(pv_path)
pv_name = p.name
pv_temp = f'{p.stem}-4dedup.txt'
pv_clean = f'{p.stem}.pkl'
# Download the file (2.3GB)
!wget -N $pv_path
# Filter for English pages, and keep just two fields: article ID (3) and monthly
# total number of page views (5). Then, remove lines with article id or page
# view values that are not a sequence of digits.
!bzcat $pv_name | grep "^en\.wikipedia" | cut -d' ' -f3,5 | grep -P "^\d+\s\d+$" > $pv_temp
# Create a Counter (dictionary) that sums up the pages views for the same
# article, resulting in a mapping from article id to total page views.
wid2pv = Counter()
with open(pv_temp, 'rt') as f:
  for line in f:
    parts = line.split(' ')
    wid2pv.update({int(parts[0]): int(parts[1])})
# write out the counter as binary file (pickle it)
with open(pv_clean, 'wb') as f:
  pickle.dump(wid2pv, f)


--2024-03-03 14:10:45--  https://dumps.wikimedia.org/other/pageview_complete/monthly/2021/2021-08/pageviews-202108-user.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.71, 2620:0:861:3:208:80:154:71
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2503235912 (2.3G) [application/octet-stream]
Saving to: ‘pageviews-202108-user.bz2’

ser.bz2              85%[================>   ]   1.99G  4.61MB/s    eta 72s    

In [ ]:
partition_postings_and_write()

In [30]:
SparkFiles.getRootDirectory()

'/hadoop/spark/tmp/spark-4b7034c5-04da-4fe2-90da-af781de92b22/userFiles-7d305c5c-39ff-483a-b319-2ab94cb55672'